# IMPORTS

## ALPHAPOSE IMPORTS

In [1]:
import argparse
import os
import platform
import sys
import time
import pickle as pkl

import numpy as np
import torch
from tqdm import tqdm
import natsort
import matplotlib
matplotlib.use('TkAgg')
from PIL import Image


alphapose_path = r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose"

if alphapose_path  not in sys.path:
    sys.path.append(alphapose_path)

from detector.apis import get_detector
from trackers.tracker_api import Tracker
from trackers.tracker_cfg import cfg as tcfg
from trackers import track
from alphapose.models import builder
from alphapose.utils.config import update_config
from alphapose.utils.detector import DetectionLoader
from alphapose.utils.file_detector import FileDetectionLoader
from alphapose.utils.transforms import flip, flip_heatmap
from alphapose.utils.vis import getTime
from alphapose.utils.webcam_detector import WebCamDetectionLoader
from alphapose.utils.writer import DataWriter

## MOTIONBERT IMPORTS

In [2]:
motionbert_path = r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\MotionBERT\MotionBERT"

if motionbert_path  not in sys.path:
    sys.path.append(motionbert_path)


import os
import numpy as np
import argparse
from tqdm import tqdm
import imageio
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from lib.utils.tools import *
from lib.utils.learning import *
from lib.utils.utils_data import flip_data
from lib.data.dataset_wild import WildDetDataset
from lib.utils.vismo import render_and_save
import os
import glob

In [3]:
import os
import random
import copy
import time
import sys
import shutil
import argparse
import errno
import math
import numpy as np
from collections import defaultdict, OrderedDict
import tensorboardX
from tqdm import tqdm
import pickle as pkl

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import StepLR

from lib.utils.tools import *
from lib.model.loss import *
from lib.model.loss_mesh import *
from lib.utils.utils_mesh import *
from lib.utils.utils_smpl import *
from lib.utils.utils_data import *
from lib.utils.learning import *
from lib.data.dataset_mesh import MotionSMPL
from lib.model.model_mesh import MeshRegressor
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import torch.multiprocessing as mp
mp.set_start_method('spawn', force=True)

import torch
import numpy as np
import glob
import os
import io
import random
import pickle
from torch.utils.data import Dataset, DataLoader
from lib.data.augmentation import Augmenter3D
from lib.utils.tools import read_pkl
from lib.utils.utils_data import flip_data, crop_scale
from lib.utils.utils_mesh import flip_thetas
from lib.utils.utils_smpl import SMPL
from torch.utils.data import Dataset, DataLoader
from lib.data.datareader_h36m import DataReaderH36M  
from lib.data.datareader_mesh import DataReaderMesh  
from lib.data.dataset_action import random_move  

# EXTRACT ALPHAPOSE COORDS

In [4]:
class Args:
    def __init__(self):
        # Demo options
        self.config = 'config.yaml'  # experiment configure file name
        self.checkpoint = 'model.pth'  # checkpoint file name
        self.sp = False  # Use single process for pytorch
        self.detector = "yolo"  # detector name
        self.detfile = ""  # detection result file
        self.inputpath = "input_images"  # image-directory
        self.inputlist = ""  # image-list
        self.inputimg = "example.jpg"  # image-name
        self.outputpath = "examples/res/"  # output-directory
        self.save_img = False  # save result as image
        self.vis = False  # visualize image
        self.showbox = False  # visualize human bbox
        self.profile = False  # add speed profiling at screen output
        self.format = None  # save in format: coco/cmu/open
        self.min_box_area = 0  # min box area to filter out
        self.detbatch = 5  # detection batch size PER GPU
        self.posebatch = 64  # pose estimation maximum batch size PER GPU
        self.eval = False  # save result json as coco format with image index
        self.gpus = "0"  # CUDA devices to use, or -1 for CPU
        self.qsize = 1024  # length of result buffer
        self.flip = False  # enable flip testing
        self.debug = False  # print detail information

        # Video options
        self.video = ""  # video-name
        self.webcam = -1  # webcam number
        self.save_video = False  # whether to save rendered video
        self.vis_fast = False  # use fast rendering

        # Tracking options
        self.pose_flow = False  # track humans in video with PoseFlow
        self.pose_track = False  # track humans in video with reid

args = Args()

args.cfg = r'E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\configs\halpe_26\resnet\256x192_res50_lr1e-3_1x_jupyter.yaml'
args.checkpoint  = r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\pretrained_models\halpe26_fast_res50_256x192.pth"
args.video = r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\3DPW\imageFiles\imageFiles\downtown_bus_00\downtown_bus_00.mp4"
args.outputpath = r'E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\test_set\downtown_bus_00'
args.save_video = False
args.vis_fast = False

cfg = update_config(args.cfg)


In [5]:
def print_finish_info():
    print('===========================> Finish Model Running.')
    if (args.save_img or args.save_video) and not args.vis_fast:
        print('===========================> Rendering remaining images in the queue...')
        print('===========================> If this step takes too long, you can enable the --vis_fast flag to use fast rendering (real-time).')

In [6]:
if platform.system() == 'Windows':
    args.sp = True

args.gpus = [int(i) for i in args.gpus.split(',')] if torch.cuda.device_count() >= 1 else [-1]
args.device = torch.device("cuda:" + str(args.gpus[0]) if args.gpus[0] >= 0 else "cpu")
args.detbatch = args.detbatch * len(args.gpus)
args.posebatch = args.posebatch * len(args.gpus)
args.tracking = args.pose_track or args.pose_flow or args.detector=='tracker'

print("Predictions will be done on device:")
print(args.device)

if not args.sp:
    torch.multiprocessing.set_start_method('forkserver', force=True)
    torch.multiprocessing.set_sharing_strategy('file_system')

Predictions will be done on device:
cuda:0


In [7]:
import os

# Initialize lists for test, train, and validation folders
test_folders = []
train_folders = []
val_folders = []

# Base directory path
base_dir = r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\3DPW\sequenceFiles\sequenceFiles"

# Subfolder names
subfolders = {
    "test": test_folders,
    "train": train_folders,
    "validation": val_folders
}

# Iterate over each subfolder
for folder_name, file_list in subfolders.items():
    folder_path = os.path.join(base_dir, folder_name)
    if os.path.exists(folder_path):  # Check if the folder exists
        for file in os.listdir(folder_path):
            if file.endswith(".pkl"):
                file_list.append(os.path.splitext(file)[0])  # Add the filename without extension

# Print the lists
print("Test folders:", test_folders)
print("Train folders:", train_folders)
print("Validation folders:", val_folders)


Test folders: ['downtown_arguing_00', 'downtown_bar_00', 'downtown_bus_00', 'downtown_cafe_00', 'downtown_car_00', 'downtown_crossStreets_00', 'downtown_downstairs_00', 'downtown_enterShop_00', 'downtown_rampAndStairs_00', 'downtown_runForBus_00', 'downtown_runForBus_01', 'downtown_sitOnStairs_00', 'downtown_stairs_00', 'downtown_upstairs_00', 'downtown_walkBridge_01', 'downtown_walking_00', 'downtown_walkUphill_00', 'downtown_warmWelcome_00', 'downtown_weeklyMarket_00', 'downtown_windowShopping_00', 'flat_guitar_01', 'flat_packBags_00', 'office_phoneCall_00', 'outdoors_fencing_01']
Train folders: ['courtyard_arguing_00', 'courtyard_backpack_00', 'courtyard_basketball_00', 'courtyard_bodyScannerMotions_00', 'courtyard_box_00', 'courtyard_capoeira_00', 'courtyard_captureSelfies_00', 'courtyard_dancing_01', 'courtyard_giveDirections_00', 'courtyard_golf_00', 'courtyard_goodNews_00', 'courtyard_jacket_00', 'courtyard_laceShoe_00', 'courtyard_rangeOfMotions_00', 'courtyard_relaxOnBench_00'

In [8]:
input_base_path = r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\3DPW\imageFiles\imageFiles"
output_base_path = r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\test_set"

# Step 1: Traverse directories and find all the .mp4 files
video_paths = glob.glob(os.path.join(input_base_path, "**", "*.mp4"), recursive=True)

In [9]:
output_train_base_path = r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\train_set"
output_test_base_path = r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\test_set"
output_val_base_path = r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\val_set"

In [10]:
#### ENTER VIDEO PATH AND OUTPUT PATH

video_paths = [r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\3DPW\imageFiles\imageFiles\downtown_arguing_00\downtown_arguing_00.mp4"]
output_dir = r'E:\Users\Philipp\Dokumente\Pose_Estimation_3D\main\output'

In [11]:
print(args)

In [12]:
import os
import glob


def initialize_models():
    """Load YOLO and Pose models once."""
    print("Loading YOLO detector...")
    yolo_detector = get_detector(args)  # Replace with actual detector initialization

    print("Loading pose model...")
    pose_model = builder.build_sppe(cfg.MODEL, preset_cfg=cfg.DATA_PRESET)
    pose_model.load_state_dict(torch.load(args.checkpoint, map_location=args.device))
    if len(args.gpus) > 1:
        pose_model = torch.nn.DataParallel(pose_model, device_ids=args.gpus).to(args.device)
    else:
        pose_model.to(args.device)
    pose_model.eval()
    print("Models loaded successfully.")
    return yolo_detector, pose_model

i_ = []

if __name__ == "__main__":
    # Load models once
    yolo_detector, pose_model = initialize_models()


    for video_path in video_paths:
        
        # # Generate the output directory structure
        # relative_path = os.path.relpath(video_path, input_base_path)
        # cur_folder = os.path.basename(os.path.dirname(video_path))
        # if cur_folder in train_folders:
        #     output_base_path = output_train_base_path
        # elif cur_folder in test_folders:
        #     output_base_path = output_test_base_path
        # elif cur_folder in output_val_base_path:
        #     output_base_path = output_val_base_path
        # else:
        #     print(f'Skipping folder {cur_folder}, because its set is not defined.')
        #     continue

        output_dir = output_dir
        os.makedirs(output_dir, exist_ok=True)
        
        # Set the output path for the specific video
        args.outputpath = output_dir
        input_source = video_path
        
        mode = "video"  # Assume videos for this example; adapt based on your needs
        print(f"Processing video: {video_path}")
        print(f'Saving in {output_base_path}')

        
        # Load detection loader for the specific video
        print('yolo IS', yolo_detector)
        det_loader = DetectionLoader(input_source, yolo_detector, cfg, args, batchSize=args.detbatch, mode=mode, queueSize=args.qsize)
        det_worker = det_loader.start()
        
        runtime_profile = {
            'dt': [],
            'pt': [],
            'pn': []
        }
        
        # Init data writer
        queueSize = 2 if mode == 'webcam' else args.qsize
        if args.save_video and mode != 'image':
            from alphapose.utils.writer import DEFAULT_VIDEO_SAVE_OPT as video_save_opt
            video_save_opt['savepath'] = os.path.join(args.outputpath, 'AlphaPose_' + os.path.basename(input_source))
            video_save_opt.update(det_loader.videoinfo)
            writer = DataWriter(cfg, args, save_video=True, video_save_opt=video_save_opt, queueSize=queueSize).start()
        else:
            writer = DataWriter(cfg, args, save_video=False, queueSize=queueSize).start()
        
        data_len = det_loader.length
        im_names_desc = tqdm(range(data_len), dynamic_ncols=True)

        batchSize = args.posebatch
        if args.flip:
            batchSize = int(batchSize / 2)
        try:
            for i in im_names_desc:
                start_time = getTime()
                with torch.no_grad():
                    (inps, orig_img, im_name, boxes, scores, ids, cropped_boxes) = det_loader.read()
                    # # Create a zero-filled tensor for keypoints
                    # # Example of printing values for debugging
                    # print("Inputs:", inps.shape)                # Print inputs
                    # print("Original Image:", orig_img.shape)    # Print the original image
                    # print("Image Name:", im_name)          # Print the image name
                    # print("Bounding Boxes:", boxes.shape)       # Print bounding boxes
                    # print("Scores:", scores.shape)               # Print scores
                    # print("IDs:", ids.shape)                     # Print object IDs
                    # print("Cropped Boxes:", cropped_boxes.shape)  # Print cropped bounding boxes
                    if orig_img is None:
                        break
                    if boxes is None or boxes.nelement() == 0:
                        print('No boxes for frame', i)
                        # Create a zero-filled tensor for keypoints
                        # Example of printing values for debugging
                        # print("Inputs:", inps)                # Print inputs
                        # print("Original Image:", orig_img)    # Print the original image
                        # print("Image Name:", im_name)          # Print the image name
                        # print("Bounding Boxes:", boxes)       # Print bounding boxes
                        # print("Scores:", scores)               # Print scores
                        # print("IDs:", ids)                     # Print object IDs
                        # print("Cropped Boxes:", cropped_boxes)  # Print cropped bounding boxes
                        # inps = torch.zeros((1, 3, 256, 192))
                        # im_name = torch.zeros((1, 3, 256, 192))
                        # boxes = torch.zeros((1, 4))  # Shape: [1, keypoints, (x, y, confidence)]
                        # scores = torch.zeros((1, 1))
                        # ids = torch.zeros((1, 1))
                        # cropped_boxes = torch.zeros((1, 4))
                        # writer.save(boxes, scores, ids, boxes, cropped_boxes, orig_img, im_name)
                        prev_scores = torch.zeros_like(prev_scores)
                        writer.save(prev_boxes, prev_scores, prev_ids, prev_hm, prev_cropped_boxes, orig_img, im_name)
                        i_.append(i)
                        continue
                    if args.profile:
                        ckpt_time, det_time = getTime(start_time)
                        runtime_profile['dt'].append(det_time)
                    # Pose Estimation
                    inps = inps.to(args.device)
                    datalen = inps.size(0)
                    leftover = 0
                    if (datalen) % batchSize:
                        leftover = 1
                    num_batches = datalen // batchSize + leftover
                    hm = []
                    if i == 1276:
                        print("num batches", num_batches)
                    for j in range(num_batches):
                        inps_j = inps[j * batchSize:min((j + 1) * batchSize, datalen)]
                        if args.flip:
                            inps_j = torch.cat((inps_j, flip(inps_j)))
                        hm_j = pose_model(inps_j)
                        if args.flip:
                            hm_j_flip = flip_heatmap(hm_j[int(len(hm_j) / 2):], pose_dataset.joint_pairs, shift=True)
                            hm_j = (hm_j[0:int(len(hm_j) / 2)] + hm_j_flip) / 2
                        hm.append(hm_j)
                        i_.append(i)
                    hm = torch.cat(hm)
                    if len(hm) == 0:
                        print('length for', im_names_desc[i], 'is zero')
                    if args.profile:
                        ckpt_time, pose_time = getTime(ckpt_time)
                        runtime_profile['pt'].append(pose_time)
                    if args.pose_track:
                        boxes, scores, ids, hm, cropped_boxes = track(tracker, args, orig_img, inps, boxes, hm, cropped_boxes, im_name, scores)
                    # if i == 614:                  
                    #     print("Boxes:", boxes)
                    #     print("Scores:", scores)
                    #     print("IDs:", ids)
                    #     print("Heatmap:", hm)
                    #     print("Cropped Boxes:", cropped_boxes)
                    #     print("Original Image:", orig_img)
                    #     print("Image Name:", im_name)
                        
                    writer.save(boxes, scores, ids, hm, cropped_boxes, orig_img, im_name)

                    prev_boxes = boxes
                    prev_scores = scores
                    prev_ids = ids
                    prev_hm = hm
                    prev_cropped_boxes = cropped_boxes
                    if args.profile:
                        ckpt_time, post_time = getTime(ckpt_time)
                        runtime_profile['pn'].append(post_time)
                
                if args.profile:
                    # TQDM
                    im_names_desc.set_description(
                        'det time: {dt:.4f} | pose time: {pt:.4f} | post processing: {pn:.4f}'.format(
                            dt=np.mean(runtime_profile['dt']), pt=np.mean(runtime_profile['pt']), pn=np.mean(runtime_profile['pn']))
                    )
                    
            print_finish_info()
            while(writer.running()):
                time.sleep(1)
                print('===========================> Rendering remaining ' + str(writer.count()) + ' images in the queue...', end='\r')
            writer.stop()
            det_loader.stop()
        except Exception as e:
            print(repr(e))
            print('An error as above occurs when processing the images, please check it')
            pass
        except KeyboardInterrupt:
            print_finish_info()
            if args.sp:
                det_loader.terminate()
                while(writer.running()):
                    time.sleep(1)
                    print('===========================> Rendering remaining ' + str(writer.count()) + ' images in the queue...', end='\r')
                writer.stop()
            else:
                det_loader.terminate()
                writer.terminate()
                writer.clear_queues()
                det_loader.clear_queues()
    
print('WARNING; STILL USING ABS PATH FOR DETECTOR')


Loading YOLO detector...
{'CONFIG': 'E:/Users/Philipp/Dokumente/Pose_Estimation_3D/Alphapose/AlphaPose/detector/yolo/cfg/yolov3-spp.cfg', 'WEIGHTS': 'E:/Users/Philipp/Dokumente/Pose_Estimation_3D/Alphapose/AlphaPose/detector/yolo/data/yolov3-spp.weights', 'INP_DIM': 608, 'NMS_THRES': 0.6, 'CONFIDENCE': 0.1, 'NUM_CLASSES': 80}
Loading pose model...


e:\miniconda-envs\main_pw3d\lib\site-packages\torchvision\models\_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
e:\miniconda-envs\main_pw3d\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Models loaded successfully.
Processing video: E:\Users\Philipp\Dokumente\Pose_Estimation_3D\3DPW\imageFiles\imageFiles\downtown_arguing_00\downtown_arguing_00.mp4
Saving in E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\test_set
yolo IS <detector.yolo_api.YOLODetector object at 0x000001DBECA7ADC8>


  0%|          | 0/898 [00:00<?, ?it/s]

Loading YOLO model..
E:/Users/Philipp/Dokumente/Pose_Estimation_3D/Alphapose/AlphaPose/detector/yolo/cfg/yolov3-spp.cfg


 69%|██████▉   | 622/898 [00:39<00:06, 42.37it/s]

HERE!


100%|██████████| 898/898 [00:45<00:00, 19.81it/s]


===========================> Finish Model Running.
Results have been written to json.ring remaining 0 images in the queue...
WARNING; STILL USING ABS PATH FOR DETECTOR


In [13]:
# Clear all variables
#%reset -f

# FILTER JSON RESULTS SO WE HAVE ONLY ONE DETECTION PER FRAME

In [14]:
import json

In [15]:
# Path to the JSON file
file_path = 'output/alphapose-results.json'

# Load the JSON file
with open(file_path, 'r') as file:
    data = json.load(file)

# Dictionary to store the first occurrence of each image_id
unique_entries = {}

# Iterate over the list of dictionaries
for entry in data:
    image_id = entry['image_id']
    # Only add the entry if the image_id is not already in the dictionary
    if image_id not in unique_entries:
        unique_entries[image_id] = entry

# Extract the unique entries as a list
filtered_data = list(unique_entries.values())

# Overwrite the file with the filtered data
with open(file_path, 'w') as file:
    json.dump(filtered_data, file, indent=4)

print(f"File '{file_path}' has been filtered and overwritten.")

File 'output/alphapose-results.json' has been filtered and overwritten.


# MOTIONBERT DETECTIONS

In [16]:
class Args:
    def __init__(self):
        # Define all your arguments as class attributes with default values
        self.config = r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\MotionBert\MotionBERT\configs\pose3d\MB_ft_h36m_global_lite.yaml"  # Path to the config file
        self.evaluate = r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\MotionBert\MotionBERT\checkpoint\pose3d\FT_MB_lite_MB_ft_h36m_global_lite\best_epoch.bin"  # Checkpoint to evaluate
        self.json_path = None  # Alphapose detection result JSON path
        self.vid_path = None  # Video path
        self.out_path = None  # Output path
        self.pixel = False  # Align with pixel coordinates
        self.focus = None  # Target person ID
        self.clip_len = 243  # Clip length for network input


In [17]:
opts = Args()
args = get_config(opts.config)


In [18]:
model_backbone = load_backbone(args)
if torch.cuda.is_available():
    model_backbone = nn.DataParallel(model_backbone)
    model_backbone = model_backbone.cuda()

print('Loading checkpoint')
checkpoint = torch.load(opts.evaluate, map_location=lambda storage, loc: storage)
model_backbone.load_state_dict(checkpoint['model_pos'], strict=True)
model_pos = model_backbone
model_pos.eval()
pass

Loading checkpoint


In [19]:
def main(opts):

    testloader_params = {
        'batch_size': 1,
        'shuffle': False,
        'num_workers': 8,
        'pin_memory': True,
        'prefetch_factor': 4,
        'persistent_workers': True,
        'drop_last': False
    }

    vid = imageio.get_reader(opts.vid_path, 'ffmpeg')
    fps_in = vid.get_meta_data()['fps']
    vid_size = vid.get_meta_data()['size']
    os.makedirs(opts.out_path, exist_ok=True)

    if opts.pixel:
        wild_dataset = WildDetDataset(opts.json_path, clip_len=opts.clip_len, vid_size=vid_size, scale_range=None, focus=opts.focus)
    else:
        wild_dataset = WildDetDataset(opts.json_path, clip_len=opts.clip_len, scale_range=[1, 1], focus=opts.focus)

    test_loader = DataLoader(wild_dataset, **testloader_params)

    results_all = []
    with torch.no_grad():
        for batch_input in tqdm(test_loader):
            N, T = batch_input.shape[:2]
            if torch.cuda.is_available():
                batch_input = batch_input.cuda()
            if args.no_conf:
                batch_input = batch_input[:, :, :, :2]
            if args.flip:
                batch_input_flip = flip_data(batch_input)
                predicted_3d_pos_1 = model_pos(batch_input)
                predicted_3d_pos_flip = model_pos(batch_input_flip)
                predicted_3d_pos_2 = flip_data(predicted_3d_pos_flip)
                predicted_3d_pos = (predicted_3d_pos_1 + predicted_3d_pos_2) / 2.0
            else:
                predicted_3d_pos = model_pos(batch_input)
            if args.rootrel:
                predicted_3d_pos[:, :, 0, :] = 0
            else:
                predicted_3d_pos[:, 0, 0, 2] = 0
            if args.gt_2d:
                predicted_3d_pos[..., :2] = batch_input[..., :2]
            results_all.append(predicted_3d_pos.cpu().numpy())

    results_all = np.hstack(results_all)
    results_all = np.concatenate(results_all)
    #render_and_save(results_all, f'{opts.out_path}/X3D.mp4', keep_imgs=False, fps=fps_in)
    if opts.pixel:
        results_all = results_all * (min(vid_size) / 2.0)
        results_all[:, :, :2] = results_all[:, :, :2] + np.array(vid_size) / 2.0
    np.save(f'{opts.out_path}/X3D.npy', results_all)

In [20]:
import os

# Define paths
video_path = video_paths[0]  # Replace with the actual video file path
json_path = "output/alphapose-results.json"  # Path to the JSON file
output_base_path = os.path.dirname(json_path)  # Save results in the same folder as the JSON file

# Overwrite existing files
overwrite = True  

# Extract video and JSON information
video_name = os.path.splitext(os.path.basename(video_path))[0]
output_path = os.path.join(output_base_path, "MotionBert", video_name)

x3d_mp4 = os.path.join(output_path, "X3D.mp4")
x3d_npy = os.path.join(output_path, "X3D.npy")

# Process the video and JSON if necessary
if overwrite or not (os.path.exists(x3d_mp4) and os.path.exists(x3d_npy)):
    print(f"Processing video: {video_name}")

    # Ensure the output directory exists
    os.makedirs(output_path, exist_ok=True)

    # Set up options for processing
    opts.vid_path = video_path
    opts.json_path = json_path
    opts.out_path = output_path

    # Call the main processing function
    main(opts)

    # Free up memory after processing
else:
    print(f"Files already exist and overwrite is set to False: {x3d_mp4}, {x3d_npy}")


Processing video: downtown_arguing_00


100%|██████████| 4/4 [00:13<00:00,  3.31s/it]


# CALCULATING SMPL PARAMETERS

In [21]:
import sys

In [22]:
smpl_params_path = r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Pose_to_SMPL\video2bvh"
if smpl_params_path not in sys.path:
    sys.path .append(smpl_params_path)


In [23]:
# from pose_estimator_2d import openpose_estimator
# from pose_estimator_3d import estimator_3d
from smpl_utils import smooth, vis, camera
from bvh_skeleton import openpose_skeleton, h36m_skeleton, cmu_skeleton

import cv2
import importlib
import numpy as np
import os
from pathlib import Path
from IPython.display import HTML
import math
#import mathutils

In [24]:
path = os.path.join(output_path, "X3D.npy")

In [25]:
data = np.load(path)

In [26]:
mapping = [(0, None), (4, None), (1, None), (7, None), (5, None), (2, None), (3, None), (6, None), (12, None), (8, None), (15, None), (None, [0, 0, 0]), (9, None), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (10, None), (13, None), (11, None), (14, None), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0])]

In [27]:
h36m_skel = h36m_skeleton.H36mSkeleton()

In [28]:
from scipy.spatial.transform import Rotation as R

In [29]:
def euler_to_quaternion_zxy(euler_z, euler_x, euler_y):
    # Create a Rotation object with 'ZXY' order
    r = R.from_euler('zxy', [euler_z, euler_x, euler_y], degrees=True)

    # Convert to quaternion
    quaternion = r.as_quat()  # Returns (x, y, z, w)

    # Return as a tuple in the desired order (W, Y, Z, X)
    return (quaternion[3], quaternion[1], quaternion[2], quaternion[0])


def rodrigues_from_pose(frame, joint_name, all_rotations):
    # Extract quaternion from input dictionary
    quat = all_rotations[frame][joint_name]  # Assumes quaternion is in (W, X, Y, Z) order

    # Create a SciPy Rotation object
    rotation = R.from_quat([quat[1], quat[2], quat[3], quat[0]])  # Convert to (X, Y, Z, W)

    # Convert to axis-angle representation
    axis = rotation.as_rotvec()  # Returns Rodrigues vector directly (axis * angle)

    return axis


def apply_mapping(B, mapping, group_size=3):
    """
    Apply a mapping to reorder and pad a list like B.
    """
    # Divide B into groups of group_size
    grouped_B = [B[i:i + group_size] for i in range(0, len(B), group_size)]

    # Apply mapping
    output_groups = []
    for index, padding in mapping:
        if index is not None:
            output_groups.append(grouped_B[index])  # Reorder from B
        elif padding is not None:
            output_groups.append(padding)  # Add padding group

    # Flatten the output groups
    reordered_B = [item for group in output_groups for item in group]
    return reordered_B


def load_and_prepare_data(data):

    # Load the 3D pose data from the .npy file
    pose_data = data

    # Get the number of frames and joints
    n_frames, n_joints, _ = pose_data.shape

    # Define the rotation matrix for -90 degrees around the x-axis
    rotation_matrix = np.array([
        [1,  0,  0],
        [0,  0, 1],
        [0,  -1,  0]
    ])

    # Apply the rotation matrix to every frame and every joint
    # We reshape the pose_data to (n_frames * n_joints, 3), apply the rotation, then reshape back
    data = np.dot(pose_data.reshape(-1, 3), rotation_matrix.T).reshape(n_frames, n_joints, 3)


    return data

def calculate_translations(channels):

    all_translations = []

    for index in range(len(channels)):
        # Initialize the dictionary to hold the results
        # Iterate through the list and extract translation values
        translation_values = channels[index][0:3]  # Extracting the translation values

        all_translations.append(translation_values)

    return all_translations


def calculate_rotations(channels, body_parts):

    all_rotations = []

    for index in range(len(channels)):
        # Initialize the dictionary to hold the results
        rotation_dict = {}
        # Iterate through the list and extract rotation values
        rotation_values = channels[index][3:]  # Extracting the rotation values, skipping the first two
        if index == 0:
            print(rotation_values)
        for i, body_part in enumerate(body_parts):
            # Take 3 consecutive values (x, y, z rotations) for each body part
            quat = euler_to_quaternion_zxy(-rotation_values[i*3+2], rotation_values[i*3+1], rotation_values[i*3])
            rotation_dict[body_part] = quat

        all_rotations.append(rotation_dict)

    return all_rotations

def calculate_all_poses(all_rotations, body_parts):

    joint_names = body_parts
    num_joints = len(joint_names)
    frame = 0
    all_poses = []


    for frame in range(len(all_rotations)):
        # Get armature pose in rodrigues representation
        pose = [0.0] * (num_joints * 3)

        for index in range(num_joints):
            joint_name = joint_names[index]
            joint_pose = rodrigues_from_pose(frame, joint_name, all_rotations)
            pose[index*3 + 0] = joint_pose[2]
            pose[index*3 + 1] = joint_pose[0]
            pose[index*3 + 2] = joint_pose[1]
        
        all_poses.append(pose)

    return all_poses

def calculate_all_thetas(all_poses):
    all_thetas = []

    for i, frame in enumerate(all_poses):
        theta_calculated = all_poses[i]

        reordered_theta = apply_mapping(theta_calculated, mapping)
        smpl_h_params = reordered_theta

        reordered_theta = reordered_theta[0:72]

        all_thetas.append(reordered_theta)
    return all_thetas

    #np.save("all_thetas_backpack.npy", all_thetas)

def save_smpl(translations, thetas, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    np.save(os.path.join(output_folder, 'all_translations.npy'), translations)
    np.save(os.path.join(output_folder, 'all_thetas.npy'), thetas)


In [30]:
def calculate_smpl_output(data):

    data = load_and_prepare_data(data)

    channels, header = h36m_skel.poses2bvh(data, output_file=None)

    all_translations = calculate_translations(channels)

    # List of body parts in the order matching the data
    body_parts = [
        'Hip', 'RightHip', 'RightKnee', 'RightAnkle', 'LeftHip', 'LeftKnee', 'LeftAnkle', 
        'Spine', 'Thorax', 'Neck', 'LeftShoulder', 'LeftElbow', 'LeftWrist', 
        'RightShoulder', 'RightElbow', 'RightWrist'
    ]

    all_rotations = calculate_rotations(channels, body_parts)

    all_poses = calculate_all_poses(all_rotations, body_parts)


    all_thetas = calculate_all_thetas(all_poses)

    return all_translations, all_thetas

In [31]:
all_translations, all_thetas = calculate_smpl_output(data)

[-8.955118798636084, 8.701388079745376, 0.0839545243032732, -6.3261396073775895, 4.813230566176596, -2.060382271891259, 0.5383237074696723, 9.274848405909221, -0.05518970066172768, 0.0, 0.0, -3.975693351829396e-16, 5.621375902514069, 8.322897743671055, 6.449372094615479, 0.24973031895028158, 7.310425286246511, -0.5426711837412692, 0.0, 0.0, -9.93923337957349e-17, 0.014359744086693058, 3.2098695109470325, 0.6649276816192868, 5.436541540770314, 7.08804992527409e-16, -2.3266151989600244e-16, 3.942120226588329, 6.6266646865177785, 1.4146543000369616, -9.290704116815983, 12.745695001072127, 85.01356417959325, -32.513705065117946, -2.3541352146956868e-15, -8.96902648485639e-15, 4.138037154422024e-33, -1.5902773407317584e-15, 2.981770013872047e-16, 22.83370801185375, 12.432785558275128, -85.20239581849702, 18.766208534534712, -5.679265551022281e-15, -5.105936804101817e-15, 0.0, 0.0, 0.0]


# SMPL PARAMS TO VERTICES

In [32]:
np.asarray(all_thetas).shape

(898, 72)

In [33]:
args.data_root = r'E:\Users\Philipp\Dokumente\Pose_Estimation_3D\MotionBert\MotionBERT\data\mesh'

In [34]:
smpl = SMPL(args.data_root, batch_size=1).double().cuda()

In [35]:
all_thetas = torch.tensor(all_thetas).to('cuda')


In [36]:
all_translations_tensor = torch.tensor(all_translations, dtype=torch.double).to('cuda')

In [37]:
output_flip_smpl = smpl(
    body_pose=all_thetas[:, 3:],
    global_orient=all_thetas[:, :3],
    pose2rot=True,
    create_transl=True,
    transl = all_translations_tensor
)

In [38]:
vid = imageio.get_reader(opts.vid_path,  'ffmpeg')
fps_in = vid.get_meta_data()['fps']
vid_size = vid.get_meta_data()['size']

In [39]:
verts_all = output_flip_smpl.vertices.detach()

verts_all = verts_all.cpu().numpy()

In [40]:
opts.vid_path = video_paths[0]

In [41]:
opts.out_path

'output\\MotionBert\\downtown_arguing_00'

In [42]:
verts_all.shape

(898, 6890, 3)

In [43]:
# # Define the rotation matrix
# R_x = np.array([
#     [1,  0,  0],
#     [0, -1,  0],
#     [0,  0, -1]
# ])

# # Assuming verts_all is already defined with shape (1262, 6890, 3)
# # Apply the rotation
# verts_all = np.einsum('ij,klj->kli', R_x, verts_all)

In [44]:
verts_all = verts_all[0:300]

In [45]:
render_and_save(verts_all, osp.join(opts.out_path, 'mesh.mp4'), keep_imgs=False, fps=fps_in, draw_face=True)

100%|██████████| 300/300 [04:19<00:00,  1.16it/s]


In [46]:
opts.out_path

'output\\MotionBert\\downtown_arguing_00'